# scVAE-Annotator: 10x Genomics Integration Demo

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/or4k2l/scVAE-Annotator/blob/main/examples/colab_10x_demo.ipynb)

This notebook demonstrates how to use scVAE-Annotator with 10x Genomics data.

## Features Demonstrated
- ✅ Loading 10x MTX format (Cell Ranger output)
- ✅ Complete annotation pipeline
- ✅ Visualization generation
- ✅ Result interpretation

**Expected Runtime**: ~8-10 minutes on Colab

**Dataset**: PBMC 3k from 10x Genomics

In [ ]:
# Install scVAE-Annotator
!git clone https://github.com/or4k2l/scVAE-Annotator.git
%cd scVAE-Annotator
!pip install -q -e .

import os
os.environ['SCIPY_ARRAY_API'] = '0'

print("✅ Installation complete!")

## Step 1: Download 10x Genomics PBMC 3k Dataset

In [ ]:
!mkdir -p data/pbmc3k
!wget -q -O data/pbmc3k/pbmc3k.tar.gz \
    http://cf.10xgenomics.com/samples/cell-exp/1.1.0/pbmc3k/pbmc3k_filtered_gene_bc_matrices.tar.gz
!cd data/pbmc3k && tar -xzf pbmc3k.tar.gz

print("✅ Dataset downloaded and extracted")

## Step 2: Load 10x MTX Format

In [ ]:
import scanpy as sc
from scvae_annotator import create_optimized_config
from scvae_annotator.tenx_loader import load_10x_data, get_10x_metadata_summary

# Load 10x data
adata = load_10x_data('data/pbmc3k/filtered_gene_bc_matrices/hg19/')

# Display metadata
summary = get_10x_metadata_summary(adata)
print("\n10x Metadata Summary:")
for key, value in summary.items():
    print(f"  {key}: {value}")

## Step 3: Run Annotation Pipeline

In [ ]:
from scvae_annotator import (
    enhanced_preprocessing,
    optimized_leiden_clustering,
    train_improved_vae,
    EnhancedAutoencoderAnnotator,
    create_visualizations
)

# Configure for Colab (faster settings)
config = create_optimized_config()
config.autoencoder_epochs = 10
config.optuna_trials = 5
config.output_dir = './results'

# Run pipeline
print("\n🔬 Preprocessing...")
adata = enhanced_preprocessing(adata, config)

print("\n🔗 Clustering...")
adata, n_clusters = optimized_leiden_clustering(adata, config)

print("\n🧠 Training VAE...")
adata = train_improved_vae(adata, config)

print("\n🏷️  Annotating...")
# Use leiden clusters as pseudo-labels for demo
adata.obs['celltype'] = adata.obs['leiden']
adata.obs['cell_type_ground_truth'] = adata.obs['leiden']

annotator = EnhancedAutoencoderAnnotator(config)
annotator.train(adata)
annotator.predict(adata)

print("\n📊 Creating visualizations...")
create_visualizations(adata, config)

print("\n✅ Pipeline complete!")

## Step 4: View Results

In [ ]:
from IPython.display import Image, display
from pathlib import Path

# Display statistics
pred_col = 'autoencoder_predictions'
conf_col = 'autoencoder_confidence'

print("📊 Results Summary:")
print(f"  Cells: {adata.n_obs}")
print(f"  Genes: {adata.n_vars}")
print(f"  Cell types: {adata.obs[pred_col].nunique()}")
print(f"  Mean confidence: {adata.obs[conf_col].mean():.3f}")

# Display plots
plots = ['umap_comparison.png', 'confusion_matrix.png', 'confidence_analysis.png']
for plot in plots:
    path = Path(config.output_dir) / plot
    if path.exists():
        print(f"\n{plot}:")
        display(Image(filename=str(path), width=700))

## 🔬 Scientific Comparison: MSE vs Poisson Likelihood

Now let's compare two modeling approaches:

1. **MSE (Mean Squared Error)**: Standard approach treating counts as continuous values
2. **Poisson Likelihood**: Statistically rigorous approach that models count data properly

The Poisson model respects the discrete, non-negative nature of scRNA-seq counts and better captures the mean-variance relationship inherent in count data.

In [ ]:
from scvae_annotator import create_scientific_config
import numpy as np

# Reload data to start fresh
adata = load_10x_data('data/pbmc3k/filtered_gene_bc_matrices/hg19/')

# Preserve raw counts for Poisson model
adata.layers['counts'] = adata.X.copy()

print("\n🔬 Training MSE Model...")
# MSE Model
config_mse = create_optimized_config()
config_mse.autoencoder_epochs = 10
config_mse.optuna_trials = 5
config_mse.output_dir = './results_mse'

adata_mse = adata.copy()
adata_mse = enhanced_preprocessing(adata_mse, config_mse)
adata_mse, n_clusters = optimized_leiden_clustering(adata_mse, config_mse)
adata_mse = train_improved_vae(adata_mse, config_mse)

adata_mse.obs['celltype'] = adata_mse.obs['leiden']
adata_mse.obs['cell_type_ground_truth'] = adata_mse.obs['leiden']
annotator_mse = EnhancedAutoencoderAnnotator(config_mse)
annotator_mse.train(adata_mse)
annotator_mse.predict(adata_mse)

print("\n🧬 Training Poisson Model...")
# Poisson Model
config_poisson = create_scientific_config()
config_poisson.autoencoder_epochs = 10
config_poisson.optuna_trials = 5
config_poisson.output_dir = './results_poisson'

adata_poisson = adata.copy()
adata_poisson = enhanced_preprocessing(adata_poisson, config_poisson)
adata_poisson, n_clusters = optimized_leiden_clustering(adata_poisson, config_poisson)
adata_poisson = train_improved_vae(adata_poisson, config_poisson)

adata_poisson.obs['celltype'] = adata_poisson.obs['leiden']
adata_poisson.obs['cell_type_ground_truth'] = adata_poisson.obs['leiden']
annotator_poisson = EnhancedAutoencoderAnnotator(config_poisson)
annotator_poisson.train(adata_poisson)
annotator_poisson.predict(adata_poisson)

print("\n✅ Both models trained successfully!")

## �� Quantitative Comparison

Let's compare the two models using key metrics:
- **Annotation Confidence**: How certain is each model?
- **Cluster Coherence**: How well do predictions align with biological structure?
- **Reconstruction Quality**: How well does each model capture the data?

In [ ]:
import pandas as pd

# Compute metrics
metrics = {
    'Model': ['MSE', 'Poisson'],
    'Mean Confidence': [
        adata_mse.obs['autoencoder_confidence'].mean(),
        adata_poisson.obs['autoencoder_confidence'].mean()
    ],
    'Min Confidence': [
        adata_mse.obs['autoencoder_confidence'].min(),
        adata_poisson.obs['autoencoder_confidence'].min()
    ],
    'High Conf. Cells (>0.9)': [
        (adata_mse.obs['autoencoder_confidence'] > 0.9).sum(),
        (adata_poisson.obs['autoencoder_confidence'] > 0.9).sum()
    ],
    'Unique Cell Types': [
        adata_mse.obs['autoencoder_predictions'].nunique(),
        adata_poisson.obs['autoencoder_predictions'].nunique()
    ]
}

df_metrics = pd.DataFrame(metrics)
print("\n📊 Model Comparison:")
print(df_metrics.to_string(index=False))

# Compute reconstruction metrics if available
if 'X_vae' in adata_mse.obsm and 'X_vae' in adata_poisson.obsm:
    from sklearn.metrics import mean_squared_error
    
    # Use normalized data for comparison
    mse_recon = mean_squared_error(
        adata_mse.X.toarray() if hasattr(adata_mse.X, 'toarray') else adata_mse.X,
        adata_mse.layers.get('X_reconstructed', adata_mse.X)
    )
    poisson_recon = mean_squared_error(
        adata_poisson.X.toarray() if hasattr(adata_poisson.X, 'toarray') else adata_poisson.X,
        adata_poisson.layers.get('X_reconstructed', adata_poisson.X)
    )
    
    print(f"\n📉 Reconstruction MSE:")
    print(f"  MSE Model: {mse_recon:.6f}")
    print(f"  Poisson Model: {poisson_recon:.6f}")

print("\n💡 Interpretation:")
print("  - Higher confidence indicates more certain predictions")
print("  - Poisson models typically show better calibrated uncertainty")
print("  - Both models should identify similar cell type structures")

## 🧬 Biological Validation with Canonical Markers

Let's validate our models using known immune cell markers:
- **CD3D/CD3E**: T cells
- **CD14/CD68**: Monocytes
- **CD79A/MS4A1**: B cells
- **NKG7/GNLY**: NK cells

In [ ]:
# Define canonical markers
markers = {
    'T cells': ['CD3D', 'CD3E'],
    'Monocytes': ['CD14', 'CD68'],
    'B cells': ['CD79A', 'MS4A1'],
    'NK cells': ['NKG7', 'GNLY']
}

print("🧬 Marker Gene Expression Analysis:\n")

for cell_type, genes in markers.items():
    print(f"{cell_type}:")
    
    for gene in genes:
        if gene in adata_mse.var_names:
            # Get expression for this gene
            mse_expr = adata_mse[:, gene].X.toarray().flatten() if hasattr(adata_mse.X, 'toarray') else adata_mse[:, gene].X.flatten()
            poisson_expr = adata_poisson[:, gene].X.toarray().flatten() if hasattr(adata_poisson.X, 'toarray') else adata_poisson[:, gene].X.flatten()
            
            print(f"  {gene}:")
            print(f"    MSE - Mean: {mse_expr.mean():.3f}, Max: {mse_expr.max():.3f}")
            print(f"    Poisson - Mean: {poisson_expr.mean():.3f}, Max: {poisson_expr.max():.3f}")
        else:
            print(f"  {gene}: Not found in dataset")
    print()

print("\n💡 Biological Interpretation:")
print("  - Look for clusters with high expression of marker genes")
print("  - Poisson models better preserve the count distribution")
print("  - Both models should identify the same biological populations")

## 📚 Why Poisson Modeling Matters### The Science Behind Count DataSingle-cell RNA-seq data are **counts** - discrete, non-negative integers representing the number of UMIs (Unique Molecular Identifiers) or reads per gene per cell. These counts follow specific statistical properties:1. **Discrete Nature**: You can't have 2.5 RNA molecules2. **Non-negative**: Counts are always ≥ 03. **Mean-Variance Relationship**: Higher expression genes show higher variance4. **Zero-inflation**: Many genes show zero counts in many cells### Why MSE Falls ShortMean Squared Error (MSE) treats counts as continuous values, which:- Ignores the discrete count nature- Assumes constant variance (homoscedasticity)- Can predict impossible negative values- Doesn't model biological zeros appropriately### Poisson AdvantagesThe Poisson likelihood:- **Respects count nature**: Models discrete, non-negative data correctly- **Captures mean-variance relationship**: Variance = mean (or with overdispersion)- **Better uncertainty quantification**: More realistic confidence estimates- **Biologically motivated**: Aligns with the molecular counting process### When It Matters MostPoisson modeling is especially important for:- Low-count genes (most genes in scRNA-seq)- Rare cell types with sparse expression- Differential expression analysis- Quantitative predictions (not just clustering)### References- Hafemeister & Satija (2019). "Normalization and variance stabilization of single-cell RNA-seq data using regularized negative binomial regression." *Genome Biology*. [DOI: 10.1186/s13059-019-1874-1](https://doi.org/10.1186/s13059-019-1874-1)- Grün et al. (2014). "Validation of noise models for single-cell transcriptomics." *Nature Methods*. [DOI: 10.1038/nmeth.2930](https://doi.org/10.1038/nmeth.2930)- Svensson et al. (2020). "Interpretable factor models of single-cell RNA-seq via variational autoencoders." *Bioinformatics*. [DOI: 10.1093/bioinformatics/btaa169](https://doi.org/10.1093/bioinformatics/btaa169)**Bottom Line**: For rigorous scientific analysis, use count-based models (Poisson/Negative Binomial) rather than treating counts as continuous data.